In [1]:
import sys

!{sys.executable} -m pip install -U  faker

  Using cached Faker-15.3.4-py3-none-any.whl (1.6 MB)
  Attempting uninstall: faker
    Found existing installation: Faker 15.3.3
    Uninstalling Faker-15.3.3:
      Successfully uninstalled Faker-15.3.3


In [2]:
import pandas as pd
import numpy as np

import datetime as dt
import time

%matplotlib inline
import matplotlib.pyplot as plt 
import seaborn as sns

## Load the Datasets into pandas dataframes

In [3]:
events_df = pd.read_csv('./data/events.csv')
category_tree_df = pd.read_csv('./data/category_tree.csv')
item_properties_1_df = pd.read_csv('./data/item_properties_part1.csv')
item_properties_2_df = pd.read_csv('./data/item_properties_part2.csv')

In [5]:
events_df

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN
...,...,...,...,...,...
2756096,1438398785939,591435,view,261427,NaN
2756097,1438399813142,762376,view,115946,NaN
2756098,1438397820527,1251746,view,78144,NaN
2756099,1438398530703,1184451,view,283392,NaN


The timestamp portion is in Unix Epoch format e.g. 1433221332117 will be converted to Tuesday, 2 June 2015 5:02:12.117 AM GMT

Visitor Id is the unique user currently browsing the website

Event is what the user is currently doing in that current timestamp

Transaction ID will only have value if the user made a purchase as shown below

In [6]:
item_properties_1_df.head()

,timestamp,itemid,property,value
0,1435460400000,460429,categoryid,1338
1,1441508400000,206783,888,1116713 960601 n277.200
2,1439089200000,395014,400,n552.000 639502 n720.000 424566
3,1431226800000,59481,790,n15360.000
4,1431831600000,156781,917,828513


Timestamp is still the same Unix / Epoch format

Item id will be the unique item identifier

Property is the Item's attributes such as category id and availability while the rest are hashed for confidentiality purposes

Value is the item's property value e.g. availability is 1 if there is stock and 0 otherwise

Note: Values that start with "n" indicate that the value preceeding it is a number e.g. n277.200 is equal to 277.2

In [7]:
category_tree_df.head()

,categoryid,parentid
0,1016,213.0
1,809,169.0
2,570,9.0
3,1691,885.0
4,536,1691.0


Category IDs explain the relationship of different products with each other e.g. Category ID 1016 is a child of Parent ID 213.

Below shows the number of items under category id 1016

In [8]:
item_properties_1_df.loc[(item_properties_1_df.property == 'categoryid') & (item_properties_1_df.value == '1016')].sort_values('timestamp').head()

,timestamp,itemid,property,value
6363096,1431226800000,339403,categoryid,1016
8597591,1431226800000,161686,categoryid,1016
7942027,1431226800000,418837,categoryid,1016
10230975,1431226800000,85538,categoryid,1016
7280176,1431226800000,278463,categoryid,1016


## Below is a snapshot of visitor id 102019 and their buying journey from viewing to transaction (purchase)

In [9]:
events_df[events_df.visitorid == 102019].sort_values('timestamp')

,timestamp,visitorid,event,itemid,transactionid
19690,1433175714335,102019,view,49521,NaN
19501,1433175801314,102019,addtocart,49521,NaN
14842,1433175812596,102019,view,150318,NaN
19573,1433175871497,102019,view,49521,NaN
8701,1433175894837,102019,view,49521,NaN
19708,1433175945872,102019,view,150318,NaN
8740,1433176042269,102019,view,49521,NaN
814,1433176736375,102019,transaction,150318,13556.0
19724,1433176736422,102019,transaction,49521,13556.0


## To simplify the problem, Let's create a Item DF with only CategoryID and merge with category_tree_df

In [10]:
item_properties_1_df.query('itemid == 49521')

,timestamp,itemid,property,value
478492,1431226800000,49521,159,519769
962850,1435460400000,49521,776,1085162
1283131,1433041200000,49521,1058,n48.000
3002504,1435460400000,49521,112,679677
3051684,1432436400000,49521,1092,291010
3317231,1432436400000,49521,761,n168.000 190776
5723948,1431226800000,49521,categoryid,1625
7099791,1433041200000,49521,839,286312
7438840,1433041200000,49521,202,222207
8058031,1433041200000,49521,463,n108.000


In [11]:
# Combine items dataframes
item_properties_df = pd.concat([item_properties_1_df, item_properties_2_df])

In [12]:
print(item_properties_1_df.shape)
print(item_properties_2_df.shape)
print(item_properties_df.shape)

(10999999, 4)
(9275903, 4)
(20275902, 4)


In [13]:
item_properties_df.apply(lambda x: len(x.unique()))

timestamp         18
itemid        417053
property        1104
value        1966868
dtype: int64

In [14]:
# Filter only the property categoryid
item_properties_df = item_properties_df[item_properties_df["property"] == "categoryid"]
print(item_properties_df.shape)

(788214, 4)


In [15]:
item_properties_df.query('itemid == 683')

,timestamp,itemid,property,value
5219307,1431831600000,683,categoryid,1147
5418090,1431226800000,683,categoryid,1147
5616873,1433041200000,683,categoryid,1147
5815656,1436670000000,683,categoryid,1147
6014439,1432436400000,683,categoryid,1147
6213222,1433646000000,683,categoryid,1147
6412005,1439089200000,683,categoryid,1147
6610788,1437879600000,683,categoryid,1147
6809571,1437274800000,683,categoryid,1147
7008354,1440903600000,683,categoryid,1244


### Since there is a time dimension on the items df, let's drop the duplicates and the time column

In [16]:
item_properties_df = item_properties_df.drop_duplicates(
                        subset = ['itemid', 'property'],
                        keep = 'last').reset_index(drop = True).drop("timestamp", axis = 1).drop('property',axis=1).rename(columns={"value": "categoryid"})

In [17]:
item_properties_df.query('itemid == 683')

,itemid,categoryid
399514,683,1147


In [18]:
category_tree_df.query('categoryid == 1147')

,categoryid,parentid
829,1147,1027.0


In [19]:
item_properties_df.dtypes

itemid         int64
categoryid    object
dtype: object

In [20]:
category_tree_df.dtypes

categoryid      int64
parentid      float64
dtype: object

In [21]:
pd.to_numeric(item_properties_df["categoryid"])
item_properties_df["categoryid"] = item_properties_df["categoryid"].apply(pd.to_numeric)

In [22]:
item_properties_df.dtypes

itemid        int64
categoryid    int64
dtype: object

In [23]:
pd.merge(item_properties_df,category_tree_df, on='categoryid')

,itemid,categoryid,parentid
0,460429,1338,1278.0
1,187386,1338,1278.0
2,358218,1338,1278.0
3,226713,1338,1278.0
4,346053,1338,1278.0
...,...,...,...
416915,432592,1548,1499.0
416916,305785,391,250.0
416917,91058,552,1120.0
416918,447347,791,NaN


# Fix events DF and brake timestamp into:  time of day,  of week, week of month, month of year

In [24]:
events_df.dtypes

timestamp          int64
visitorid          int64
event             object
itemid             int64
transactionid    float64
dtype: object

In [25]:
events_df.sort_values('timestamp').head()

,timestamp,visitorid,event,itemid,transactionid
1462974,1430622004384,693516,addtocart,297662,NaN
1464806,1430622011289,829044,view,60987,NaN
1463000,1430622013048,652699,view,252860,NaN
1465287,1430622024154,1125936,view,33661,NaN
1462955,1430622026228,693516,view,297662,NaN


In [26]:
events_df["timestamp"] = pd.to_datetime(events_df["timestamp"], unit='ms')

In [27]:
events_df = events_df.drop("transactionid", axis=1)

In [28]:
events_df

,timestamp,visitorid,event,itemid
0,2015-06-02 05:02:12.117,257597,view,355908
1,2015-06-02 05:50:14.164,992329,view,248676
2,2015-06-02 05:13:19.827,111016,view,318965
3,2015-06-02 05:12:35.914,483717,view,253185
4,2015-06-02 05:02:17.106,951259,view,367447
...,...,...,...,...
2756096,2015-08-01 03:13:05.939,591435,view,261427
2756097,2015-08-01 03:30:13.142,762376,view,115946
2756098,2015-08-01 02:57:00.527,1251746,view,78144
2756099,2015-08-01 03:08:50.703,1184451,view,283392


In [29]:
events_df["month"] = events_df["timestamp"].dt.month
events_df["weekday"] = events_df["timestamp"].dt.weekday
events_df['period'] = (events_df['timestamp'].dt.hour % 24 + 4) // 4
events_df['period'].replace({1: 'Late Night',
                      2: 'Early Morning',
                      3: 'Morning',
                      4: 'Noon',
                      5: 'Evening',
                      6: 'Night'}, inplace=True)

In [30]:
events_df.drop("timestamp", axis=1, inplace=True)

In [31]:
events_df.apply(lambda x: len(x.unique()))

visitorid    1407580
event              3
itemid        235061
month              5
weekday            7
period             6
dtype: int64

In [37]:
events_df.head(2)

,visitorid,event,itemid,month,weekday,period
0,257597,view,355908,6,1,Early Morning
1,992329,view,248676,6,1,Early Morning


# Let's create a fake users table with information about their state

In [32]:
from faker import Faker
fake = Faker()

In [33]:
users =[]
states=[]

for user in events_df["visitorid"].unique():
    users.append(user)
    states.append(fake.state())

In [34]:
users_df = pd.DataFrame.from_dict({'visitorid':users, 'state':states})

In [35]:
users_df.head()

,visitorid,state
0,257597,Nevada
1,992329,Colorado
2,111016,Utah
3,483717,Utah
4,951259,Alabama


## Save final datasets to disk

In [36]:
events_df.to_csv('./data/events_final.csv', index=False)
item_properties_df.to_csv('./data/items_final.csv', index=False)
users_df.to_csv('./data/users_final.csv', index=False)